In [95]:
# !pip install xgboost
# !pip install lightgbm
# !pip install imblearn
# !pip install catboost


In [96]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.svm import SVC
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from catboost import CatBoostClassifier

In [97]:
df = pd.read_csv("data.csv")
df.drop(columns=["id"], inplace=True)
df.sample(5)
# df.describe


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
445,B,11.99,24.89,77.61,441.3,0.10300,0.09218,0.05441,0.04274,0.1820,...,12.98,30.36,84.48,513.9,0.1311,0.1822,0.1609,0.12020,0.2599,0.08251
145,B,11.90,14.65,78.11,432.8,0.11520,0.12960,0.03710,0.03003,0.1995,...,13.15,16.51,86.26,509.6,0.1424,0.2517,0.0942,0.06042,0.2727,0.10360
426,B,10.48,14.98,67.49,333.6,0.09816,0.10130,0.06335,0.02218,0.1925,...,12.13,21.57,81.41,440.4,0.1327,0.2996,0.2939,0.09310,0.3020,0.09646
221,B,13.56,13.90,88.59,561.3,0.10510,0.11920,0.07860,0.04451,0.1962,...,14.98,17.13,101.10,686.6,0.1376,0.2698,0.2577,0.09090,0.3065,0.08177
400,M,17.91,21.02,124.40,994.0,0.12300,0.25760,0.31890,0.11980,0.2113,...,20.80,27.78,149.60,1304.0,0.1873,0.5917,0.9034,0.19640,0.3245,0.11980


In [98]:
skewness = df.skew(numeric_only=True)
high_skew_cols = skewness[abs(skewness) > 1].index.tolist()
print(high_skew_cols)
# skew_df = pd.DataFrame({
#     "column": df.select_dtypes(include=['int64', 'float64']).columns,
#     "skewness": df.skew(numeric_only=True).values
# })
# high_skew = skew_df[abs(skew_df['skewness']) > 1]
# print(high_skew)

['area_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'perimeter_worst', 'area_worst', 'compactness_worst', 'concavity_worst', 'symmetry_worst', 'fractal_dimension_worst']


In [ ]:
X = df.drop("diagnosis", axis=1)
y = df["diagnosis"]
y = y.replace({'B':0, 'M':1})
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=780, shuffle=True)
rest_cols = X.drop(high_skew_cols, axis=1)
# dropping these columns for further minmax scaling because their scale is too high
rest_cols.drop(columns=['radius_mean', 'texture_mean', 'perimeter_mean', 'texture_worst'], axis=1, inplace=True)
X.columns

C:\Users\LOQ\AppData\Local\Temp\ipykernel_21520\3792016441.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({'B':0, 'M':1})


Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [100]:
plt.figure(figsize=(15,10))
corr = X.corr()
# sns.heatmap(corr)

<Figure size 1500x1000 with 0 Axes>

In [101]:
# Applying column transformer & Function transformer for high skewed features

trf = ColumnTransformer(transformers=[
	('log_transform', FunctionTransformer(np.log1p), high_skew_cols),
	('minmax_scaler', MinMaxScaler(), ['radius_mean', 'texture_mean', 'perimeter_mean', 'texture_worst']),
	('std_scaler', StandardScaler(), rest_cols.columns.tolist())

], remainder='passthrough')

In [102]:
# Now initializing the models
rfc = RandomForestClassifier(n_estimators=150, criterion='gini', max_depth=5, min_samples_split=10, max_features='sqrt', n_jobs=-1, class_weight='balanced')
# Assigning class weights
# weights = np.where(y_train == 'M', 5, 1)
gbc = GradientBoostingClassifier(
  	n_estimators=200,
	learning_rate=0.1, 
	max_depth=5, 
	min_samples_split=10,
	max_features='sqrt', 
	criterion='friedman_mse', 
	random_state=1025
)
# svc = SVC()
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.2,
    max_depth=6, #10
    subsample=0.7,
    colsample_bytree=0.6,
    min_child_weight=1,
    gamma=0.1,
    reg_alpha=0,
    reg_lambda=1,
    tree_method="hist",
    eval_metric="logloss"
)
# {'model__subsample': 0.7, 'model__n_estimators': 300, 'model__min_child_weight': 1, 'model__max_depth': 10, 'model__learning_rate': 0.2, 'model__gamma': 0.1, 'model__colsample_bytree': 0.6}


cbc = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.05, verbose=0)


MOST IMP FEATURES: ['perimeter_worst', 'area_worst', 'concave points_worst', 'concave points_mean', 'concavity_worst', 'area_mean', 'texture_worst', 'radius_worst', 'concavity_mean', 'smoothness_mean']

In [103]:
pipeline = Pipeline([
	('trf', trf),
	# ('smote', SMOTE()),
	('model', xgb)
])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
print(scores.mean())

0.971852196863841


In [110]:
import joblib

# Save pipeline components separately
joblib.dump(pipeline.named_steps['model'], 'model.pkl')
joblib.dump(pipeline.named_steps['trf'], 'scaler.pkl')

print("Model and scaler saved!")

Model and scaler saved!


In [104]:
print(confusion_matrix(y_test, y_pred)) # no, yes 
print(recall_score(y_test, y_pred, pos_label=1))

[[71  2]
 [ 2 39]]
0.9512195121951219


In [105]:
print(classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.973     0.973     0.973        73
           1      0.951     0.951     0.951        41

    accuracy                          0.965       114
   macro avg      0.962     0.962     0.962       114
weighted avg      0.965     0.965     0.965       114



In [106]:
# from sklearn.metrics import ConfusionMatrixDisplay

# disp = ConfusionMatrixDisplay.from_predictions(
#     y_test, 
#     y_pred, 
#     display_labels=['B (Benign)', 'M (Malignant)']
# )
# plt.title("Confusion Matrix")
# plt.show()


In [107]:
# # For XGBoost Hyperparameter Tunning
# from sklearn.model_selection import RandomizedSearchCV

# # Define the parameter grid for XGBoost
# # Note: 'model' is the name of the XGB step in the pipeline
# param_dist_xgb = {
#     'model__n_estimators': [100, 200, 300, 400, 500],
#     'model__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
#     'model__max_depth': [3, 4, 5, 6, 8, 10],
#     'model__min_child_weight': [1, 3, 5, 7],
#     'model__gamma': [0, 0.1, 0.2, 0.3, 0.4],
#     'model__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
#     'model__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
# }

# # Initialize RandomizedSearchCV
# random_search_xgb = RandomizedSearchCV(
#     estimator=pipeline,
#     param_distributions=param_dist_xgb,
#     n_iter=50,
#     scoring='accuracy',
#     cv=5,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# # Fit the random search model
# random_search_xgb.fit(X_train, y_train)

# # Print results
# print("Best Parameters found: ", random_search_xgb.best_params_)
# print("Best Accuracy Score: ", random_search_xgb.best_score_)

# # Evaluate
# best_xgb = random_search_xgb.best_estimator_
# y_pred_xgb = best_xgb.predict(X_test)
# print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_xgb))
# print("Recall Score:", recall_score(y_test, y_pred_xgb, pos_label=1))

In [108]:
# # Define the parameter grid for CatBoost
# # Note: 'model' is the name of the CatBoost step in the pipeline
# param_dist_cbc = {
#     'model__iterations': [100, 200, 300, 500],
#     'model__learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2],
#     'model__depth': [4, 6, 8, 10],
#     'model__l2_leaf_reg': [1, 3, 5, 7, 9],
#     'model__border_count': [32, 50, 100, 254],
#     'model__bagging_temperature': [0, 1],
#     'model__random_strength': [1, 5, 10]
# }

# # Initialize RandomizedSearchCV
# random_search_cbc = RandomizedSearchCV(
#     estimator=pipeline,
#     param_distributions=param_dist_cbc,
#     n_iter=50,
#     scoring='accuracy',
#     cv=5,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# # Fit the random search model
# random_search_cbc.fit(X_train, y_train)

# # Print results
# print("Best Parameters found: ", random_search_cbc.best_params_)
# print("Best Accuracy Score: ", random_search_cbc.best_score_)

# # Evaluate
# best_cbc = random_search_cbc.best_estimator_
# y_pred_cbc = best_cbc.predict(X_test)
# print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_cbc))
# print("Recall Score:", recall_score(y_test, y_pred_cbc, pos_label=1))

In [111]:
# Function for getting most important 10 features
def getimpfeatures():
	# Access the model and preprocessor steps
	model_step = pipeline.named_steps['model']

	# Reconstruct the feature names list based on the transformer order in your pipeline
	feature_names = list(high_skew_cols) + \
					['radius_mean', 'texture_mean', 'perimeter_mean', 'texture_worst'] + \
					list(rest_cols.columns)

	# Get feature importances
	if hasattr(model_step, 'feature_importances_'):
		importances = model_step.feature_importances_
		
		# Create a DataFrame to sort them
		feat_imp = pd.DataFrame({
			'feature': feature_names,
			'importance': importances
		})

		# Get the top 10 features sorted by importance
		top_10_df = feat_imp.sort_values(by='importance', ascending=False).head(10)
		
		# Extract just the list of names
		top_10_features = top_10_df['feature'].tolist()
		
		return top_10_features
		
	else:
		print(f"The model {type(model_step).__name__} does not expose feature_importances_.")

In [112]:
getimpfeatures()

['perimeter_worst',
 'area_worst',
 'concave points_worst',
 'concave points_mean',
 'concavity_worst',
 'area_mean',
 'texture_worst',
 'radius_worst',
 'concavity_mean',
 'smoothness_mean']